In [1]:
import pandas as pd
import numpy as np
from itertools import product

from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score,
    precision_recall_curve, roc_auc_score, make_scorer, f1_score, recall_score
)
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_predict, RandomizedSearchCV

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    RandomForestClassifier, VotingClassifier, StackingClassifier
)
import xgboost as xgb
import lightgbm as lgb

from sklearn.calibration import CalibratedClassifierCV

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split


# Tianhan Baseline bagging + neural network

In [2]:
class SimpleNN(nn.Module):
    def __init__(self, input_dim):
        super(SimpleNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 10),
            nn.ReLU(),
            nn.Linear(10, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [3]:
numeric_df = pd.read_csv("data/cleaned_data.csv")
numeric_df = numeric_df.drop(columns = ["Unnamed: 0"])
X = numeric_df.drop(['FraudFound_P', 'PolicyNumber'], axis=1)
y = numeric_df['FraudFound_P']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [4]:
# Bagging
def create_bagged_sets(X, y, n=10):
    X_sets, y_sets = [], []
    for _ in range(n):
        idx = np.random.choice(len(X), size=len(X), replace=True)
        X_sets.append(X.iloc[idx])
        y_sets.append(y.iloc[idx])
    return X_sets, y_sets

X_sets, y_sets = create_bagged_sets(X_train, y_train)

In [5]:
# Train 10 of each model
def train_models(X_sets, y_sets, model_type):
    models = []
    for i in range(10):
        if model_type == 'lr':
            model = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000))
        elif model_type == 'dt':
            model = DecisionTreeClassifier()
        elif model_type == 'rf':
            model = RandomForestClassifier(n_estimators=100, n_jobs=-1)
        model.fit(X_sets[i], y_sets[i])
        models.append(model)
    return models

lr_models = train_models(X_sets, y_sets, 'lr')
dt_models = train_models(X_sets, y_sets, 'dt')
rf_models = train_models(X_sets, y_sets, 'rf')

In [6]:
# Generate stacked prediction probabilities for each base model in the ensemble
# For each model in the list, predict the probability of the positive class (fraud = 1)
# The result is a 2D array where each column represents predictions from one model
# This stacked array will be used as input features to the neural network ensemble

def get_stacked_preds(models, X):
    preds = [model.predict_proba(X)[:,1] for model in models]
    return np.column_stack(preds)

lr_preds_train = get_stacked_preds(lr_models, X_train)
dt_preds_train = get_stacked_preds(dt_models, X_train)
rf_preds_train = get_stacked_preds(rf_models, X_train)


In [7]:
# Train a simple PyTorch neural network on the stacked predictions of base models
# This NN learns to ensemble the outputs from 10 base models for a specific method (LR, DT, or RF)

def train_nn_model(X_np, y_np, input_dim, epochs=10, lr=0.001):
    model = SimpleNN(input_dim)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    dataset = TensorDataset(torch.tensor(X_np, dtype=torch.float32),
                            torch.tensor(y_np.values, dtype=torch.float32).unsqueeze(1))
    loader = DataLoader(dataset, batch_size=64, shuffle=True)

    model.train()
    for epoch in range(epochs):
        for xb, yb in loader:
            optimizer.zero_grad()
            output = model(xb)
            loss = criterion(output, yb)
            loss.backward()
            optimizer.step()
    return model

lr_nn = train_nn_model(lr_preds_train, y_train, lr_preds_train.shape[1])
dt_nn = train_nn_model(dt_preds_train, y_train, dt_preds_train.shape[1])
rf_nn = train_nn_model(rf_preds_train, y_train, rf_preds_train.shape[1])

In [8]:
# Generate the output predictions from a trained PyTorch neural network
# This is used to obtain ensemble predictions from each of the LR, DT, RF ensembles
# These outputs will serve as input features to the final-level ensemble model

def get_nn_output(model, X_np):
    model.eval()
    with torch.no_grad():
        input_tensor = torch.tensor(X_np, dtype=torch.float32)
        return model(input_tensor).numpy()

lr_out_train = get_nn_output(lr_nn, lr_preds_train)
dt_out_train = get_nn_output(dt_nn, dt_preds_train)
rf_out_train = get_nn_output(rf_nn, rf_preds_train)

stacked_train = np.hstack([lr_out_train, dt_out_train, rf_out_train])
final_nn = train_nn_model(stacked_train, y_train, 3)


In [9]:
# Stacked test preds
lr_preds_test = get_stacked_preds(lr_models, X_test)
dt_preds_test = get_stacked_preds(dt_models, X_test)
rf_preds_test = get_stacked_preds(rf_models, X_test)

lr_out_test = get_nn_output(lr_nn, lr_preds_test)
dt_out_test = get_nn_output(dt_nn, dt_preds_test)
rf_out_test = get_nn_output(rf_nn, rf_preds_test)

stacked_test = np.hstack([lr_out_test, dt_out_test, rf_out_test])
final_preds = get_nn_output(final_nn, stacked_test)
final_preds_binary = (final_preds > 0.5).astype(int)

# Metrics
print(classification_report(y_test, final_preds_binary))
print("ROC AUC Score:", roc_auc_score(y_test, final_preds))

              precision    recall  f1-score   support

           0       0.94      0.99      0.97      2899
           1       0.33      0.09      0.14       185

    accuracy                           0.93      3084
   macro avg       0.64      0.54      0.55      3084
weighted avg       0.91      0.93      0.92      3084

ROC AUC Score: 0.5558515051788595


## Fine tuning of models
Here, we tried hyperparameter tuning for number of hiddne units, epochs, learning rate, dropout rate
Class imbalance handling using pos_weight in loss function
Threshold optimization instead of using a fixed 0.5


In [10]:
def train_nn_model(X_np, y_np, input_dim, epochs=20, lr=0.001, hidden_dim=10, dropout=0.0):
    class TunedNN(nn.Module):
        def __init__(self, input_dim, hidden_dim, dropout):
            super().__init__()
            self.model = nn.Sequential(
                nn.Linear(input_dim, hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(hidden_dim, 1)  # No Sigmoid here
            )

        def forward(self, x):
            return self.model(x)

    model = TunedNN(input_dim, hidden_dim, dropout)

    # Handle class imbalance
    fraud_ratio = y_np.value_counts()[0] / y_np.value_counts()[1]
    criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([fraud_ratio], dtype=torch.float32))

    optimizer = optim.Adam(model.parameters(), lr=lr)

    dataset = TensorDataset(
        torch.tensor(X_np, dtype=torch.float32),
        torch.tensor(y_np.values, dtype=torch.float32).unsqueeze(1)
    )
    loader = DataLoader(dataset, batch_size=64, shuffle=True)

    model.train()
    for epoch in range(epochs):
        for xb, yb in loader:
            optimizer.zero_grad()
            output = model(xb)
            loss = criterion(output, yb)
            loss.backward()
            optimizer.step()
    return model


def evaluate_thresholds(y_true, y_prob):
    best_f1 = 0
    best_threshold = 0.5
    best_metrics = {}

    for t in np.arange(0.05, 0.95, 0.01):
        y_pred = (y_prob > t).astype(int)
        f1 = f1_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        auc = roc_auc_score(y_true, y_prob)

        if f1 > best_f1 or (f1 == best_f1 and recall > best_metrics.get("recall", 0)):
            best_f1 = f1
            best_threshold = t
            best_metrics = {
                "threshold": t,
                "f1": f1,
                "recall": recall,
                "auc": auc,
                "confusion_matrix": confusion_matrix(y_true, y_pred)
            }

    return best_metrics


In [11]:
# Get outputs from test set
lr_preds_test = get_stacked_preds(lr_models, X_test)
dt_preds_test = get_stacked_preds(dt_models, X_test)
rf_preds_test = get_stacked_preds(rf_models, X_test)

lr_out_test = get_nn_output(lr_nn, lr_preds_test)
dt_out_test = get_nn_output(dt_nn, dt_preds_test)
rf_out_test = get_nn_output(rf_nn, rf_preds_test)

stacked_test = np.hstack([lr_out_test, dt_out_test, rf_out_test])
final_test_probs = get_nn_output(final_nn, stacked_test)

# Tune threshold and evaluate
best_metrics = evaluate_thresholds(y_test, final_test_probs)

print(f"Best Threshold: {best_metrics['threshold']:.2f}")
print(f"F1 Score: {best_metrics['f1']:.4f}")
print(f"Recall: {best_metrics['recall']:.4f}")
print(f"ROC AUC: {best_metrics['auc']:.4f}")
print("Confusion Matrix:")
print(best_metrics["confusion_matrix"])

Best Threshold: 0.05
F1 Score: 0.2204
Recall: 0.2216
ROC AUC: 0.5559
Confusion Matrix:
[[2753  146]
 [ 144   41]]


In [12]:
## Proceed to do grid search
# Define the search space
param_grid = {
    'hidden_dim': [8, 16, 32],
    'dropout': [0.0, 0.2, 0.5],
    'lr': [0.001, 0.0005],
    'epochs': [20, 40]
}

# Generate all combinations of parameters
param_combos = list(product(
    param_grid['hidden_dim'],
    param_grid['dropout'],
    param_grid['lr'],
    param_grid['epochs']
))

best_result = {
    'f1': 0,
    'recall': 0,
    'auc': 0,
    'params': None,
    'threshold': 0.5,
    'confusion_matrix': None
}

In [13]:
def get_nn_output_finetuning(model, X_np):
    model.eval()
    with torch.no_grad():
        input_tensor = torch.tensor(X_np, dtype=torch.float32)
        logits = model(input_tensor)
        probs = torch.sigmoid(logits)  # Manual sigmoid
        return probs.numpy()

In [14]:
for hidden_dim, dropout, lr, epochs in param_combos:
    # Train NN on training set
    model = train_nn_model(
        stacked_train, y_train,
        input_dim=3,
        hidden_dim=hidden_dim,
        dropout=dropout,
        lr=lr,
        epochs=epochs
    )

    # Get probabilities on test set
    final_test_probs = get_nn_output_finetuning(model, stacked_test)

    # Evaluate at best threshold
    metrics = evaluate_thresholds(y_test, final_test_probs)

    if (metrics['f1'] > best_result['f1']) or \
       (metrics['f1'] == best_result['f1'] and metrics['recall'] > best_result['recall']):
        best_result.update({
            'f1': metrics['f1'],
            'recall': metrics['recall'],
            'auc': metrics['auc'],
            'params': {
                'hidden_dim': hidden_dim,
                'dropout': dropout,
                'lr': lr,
                'epochs': epochs
            },
            'threshold': metrics['threshold'],
            'confusion_matrix': metrics['confusion_matrix']
        })

In [15]:
print("Best Hyperparameters:")
print(best_result['params'])
print(f"Optimal Threshold: {best_result['threshold']:.2f}")
print(f"F1 Score: {best_result['f1']:.4f}")
print(f"Recall: {best_result['recall']:.4f}")
print(f"ROC AUC: {best_result['auc']:.4f}")
print("Confusion Matrix:")
print(best_result['confusion_matrix'])


Best Hyperparameters:
{'hidden_dim': 8, 'dropout': 0.0, 'lr': 0.0005, 'epochs': 20}
Optimal Threshold: 0.24
F1 Score: 0.2729
Recall: 0.4757
ROC AUC: 0.7369
Confusion Matrix:
[[2527  372]
 [  97   88]]


In [16]:
final_nn_best = train_nn_model(
    stacked_train, y_train,
    input_dim=3,
    hidden_dim=8,
    dropout=0.2,
    lr=0.001,
    epochs=20
)

final_probs = get_nn_output_finetuning(final_nn_best, stacked_test)

final_preds = (final_probs > 0.07).astype(int)

In [17]:
print(classification_report(y_test, final_preds))
print("Confusion Matrix:")
print(confusion_matrix(y_test, final_preds))
print("ROC AUC:", roc_auc_score(y_test, final_probs))

              precision    recall  f1-score   support

           0       0.95      0.94      0.95      2899
           1       0.21      0.24      0.23       185

    accuracy                           0.90      3084
   macro avg       0.58      0.59      0.59      3084
weighted avg       0.91      0.90      0.90      3084

Confusion Matrix:
[[2733  166]
 [ 140   45]]
ROC AUC: 0.8220747135545343


# Ashley's Voting and Stacking model ensemble

In [8]:
numeric_df = pd.read_csv("data/cleaned_data.csv")
numeric_df = numeric_df.drop(columns = ["Unnamed: 0"])
X = numeric_df.drop(['FraudFound_P', 'PolicyNumber'], axis=1)
y = numeric_df['FraudFound_P']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # 80-20 split


In [13]:
smote = SMOTE(random_state=42)
X_train_oversampled, y_train_oversampled = smote.fit_resample(X_train, y_train)

df_smote = pd.concat([X_train_oversampled, y_train_oversampled], axis = 1)
X_smote = df_smote.drop(columns="FraudFound_P")
y_smote = df_smote["FraudFound_P"]

df_train_smote = pd.concat([X_smote, y_smote], axis=1)
df_test = pd.concat([X_test, y_test], axis=1)


In [14]:
X_train_oversampled

,Month,WeekOfMonth,DayOfWeek,AccidentArea,MonthClaimed,WeekOfMonthClaimed,DayOfWeekClaimed,Sex,Age,Fault,...,BasePolicy,UnusualDeductible,ClaimsPerAgent,MaritalStatus,PriceToAgeRatio,PriceToAgeRatio_Capped,HighRiskVehicleFlag,ClaimAmountRisk,DaysBetween,ClaimProcessingTime
0,5,2,5,1,5,4,2,1,42,0,...,1,0,15179,1,0.251314,0.251314,0,1,1,376
1,11,4,6,1,11,4,1,1,33,1,...,1,0,15179,1,0.251314,0.251314,0,1,1,360
2,3,1,4,1,3,3,4,0,40,0,...,2,0,15179,1,0.251314,0.251314,0,1,1,379
3,4,4,7,1,4,4,1,0,29,0,...,2,0,15179,0,0.251314,0.251314,0,4,1,359
4,3,5,3,1,3,5,3,1,72,0,...,1,0,15179,1,0.223144,0.223144,0,2,1,365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23217,1,2,5,0,2,2,2,1,52,0,...,3,0,15179,1,0.117783,0.117783,0,0,1,377
23218,4,3,5,0,4,3,2,0,33,0,...,2,0,15179,0,0.326930,0.326930,0,1,1,361
23219,3,3,4,1,4,1,2,0,30,0,...,3,0,15179,0,0.320814,0.320814,0,5,1,372
23220,5,3,2,1,5,3,4,1,39,0,...,1,0,15179,1,0.315845,0.315845,0,3,1,368


In [15]:
calibrated_rf = CalibratedClassifierCV(
    RandomForestClassifier(n_estimators=100, class_weight="balanced", random_state=36),
    method='isotonic', cv=3
)

voting_clf = VotingClassifier(
    estimators=[
        ('lr', make_pipeline(StandardScaler(), LogisticRegression(max_iter=2000, class_weight="balanced"))),
        ('rf', calibrated_rf),
        ('xgb', xgb.XGBClassifier(n_estimators=100, eval_metric="logloss", random_state=42, scale_pos_weight=12)),
        ('lgb', lgb.LGBMClassifier(n_estimators=100, class_weight='balanced', random_state=42))
    ],
    voting='soft'  # Change to 'hard' for majority voting
)

# Train voting classifier
voting_clf.fit(X_train_oversampled, y_train_oversampled)

# Predict
y_proba = voting_clf.predict_proba(X_test)[:, 1]
y_pred_voting = (y_proba >= 0.30).astype(int)
# Evaluate
print("==== Voting Classifier ====")
print(f"Accuracy: {accuracy_score(y_test, y_pred_voting):.4f}")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_voting))
print("Classification Report:")
print(classification_report(y_test, y_pred_voting))
roc_auc = roc_auc_score(y_test, y_proba)
print(f"ROC AUC Score: {roc_auc:.4f}")

[LightGBM] [Info] Number of positive: 11611, number of negative: 11611
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001137 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1098
[LightGBM] [Info] Number of data points in the train set: 23222, number of used features: 38
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
==== Voting Classifier ====
Accuracy: 0.8119
Confusion Matrix:
[[2401  484]
 [  96  103]]
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.83      0.89      2885
           1       0.18      0.52      0.26       199

    accuracy                           0.81      3084
   macro avg       0.57      0.67      0.58      3084
weighted avg       0.91      0.81      0.85      3084

ROC AUC Score: 0.7987


In [16]:
calibrated_rf = CalibratedClassifierCV(
    RandomForestClassifier(n_estimators=100, class_weight="balanced", random_state=36),
    method='isotonic', cv=3
)

voting_clf = VotingClassifier(
    estimators=[
        ('lr', make_pipeline(StandardScaler(), LogisticRegression(max_iter=2000, class_weight="balanced"))),
        ('rf', calibrated_rf),
        ('xgb', xgb.XGBClassifier(n_estimators=100, eval_metric="logloss", random_state=42, scale_pos_weight=12)),
        ('lgb', lgb.LGBMClassifier(n_estimators=100, class_weight='balanced', random_state=42))
    ],
    voting='soft'  # Change to 'hard' for majority voting
)

# Train voting classifier
voting_clf.fit(X_train_oversampled, y_train_oversampled)

# Predict
y_proba = voting_clf.predict_proba(X_test)[:, 1]
from sklearn.metrics import precision_recall_curve, f1_score, roc_auc_score

precisions, recalls, thresholds = precision_recall_curve(y_test, y_proba)

f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-6)
best_threshold = thresholds[f1_scores.argmax()]
print(f"Best Threshold by F1: {best_threshold:.2f}, F1-score: {f1_scores.max():.3f}")

roc_auc = roc_auc_score(y_test, y_proba)
print(f"ROC AUC Score: {roc_auc:.4f}")

# Use the new threshold
y_pred_best = (y_proba >= best_threshold).astype(int)
print(classification_report(y_test, y_pred_best, digits=3))

# Model Update Summary (Voting Classifier)

# - Applied class balancing techniques: 'class_weight' and 'scale_pos_weight' for base models
# - Calibrated the Random Forest classifier using isotonic regression for better probability estimates
# - Used 'soft' voting to average predicted probabilities across models
# - Tuned the classification threshold using precision-recall curve to optimize for F1-score of fraud class (label=1)
# ➤ These changes aim to improve fraud detection performance, especially recall and F1, on the imbalanced test set

[LightGBM] [Info] Number of positive: 11611, number of negative: 11611
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001044 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1098
[LightGBM] [Info] Number of data points in the train set: 23222, number of used features: 38
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Best Threshold by F1: 0.27, F1-score: 0.266
ROC AUC Score: 0.7987
              precision    recall  f1-score   support

           0      0.966     0.802     0.877      2885
           1      0.172     0.593     0.266       199

    accuracy                          0.789      3084
   macro avg      0.569     0.698     0.571      3084
weighted avg      0.915     0.789     0.837      3084



In [17]:
# Logistic Regression with StandardScaler
lr_model = make_pipeline(
    StandardScaler(),
    LogisticRegression(max_iter=2000, class_weight='balanced', random_state=42)
)

# Random Forest
rf_model = RandomForestClassifier(class_weight='balanced', random_state=42)

# XGBoost with imbalance handling
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', scale_pos_weight=12, random_state=42)

# LightGBM
lgb_model = lgb.LGBMClassifier(class_weight='balanced', random_state=42)

voting_clf = VotingClassifier(
    estimators=[
        ('lr', lr_model),
        ('rf', rf_model),
        ('xgb', xgb_model),
        ('lgb', lgb_model)
    ],
    voting='soft'
)

pipeline = Pipeline([
    ('smote', SMOTE(random_state=42)),
    ('voting', voting_clf)
])

param_dist = {
    'voting__rf__n_estimators': [100, 200, 300],
    'voting__rf__max_depth': [5, 10, None],
    'voting__xgb__learning_rate': [0.01, 0.1, 0.2],
    'voting__xgb__n_estimators': [100, 200],
    'voting__lgb__n_estimators': [100, 200],
    'voting__lgb__learning_rate': [0.01, 0.1]
}

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scorer = make_scorer(f1_score, pos_label=1)

random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_dist,
    n_iter=10,
    scoring=scorer,
    cv=skf,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# Fit on training data
random_search.fit(X_train, y_train)

# Best model results
print("Best F1-score (fraud class):", random_search.best_score_)
print("Best parameters:")
print(random_search.best_params_)

from sklearn.metrics import classification_report, confusion_matrix
roc_auc = roc_auc_score(y_test, y_proba)
print(f"ROC AUC Score: {roc_auc:.4f}")

y_pred_best = random_search.predict(X_test)
print(confusion_matrix(y_test, y_pred_best))
print(classification_report(y_test, y_pred_best, digits=3))

Fitting 5 folds for each of 10 candidates, totalling 50 fits


C:\Users\xutia\AppData\Roaming\Python\Python313\site-packages\xgboost\training.py:183: UserWarning: [21:32:40] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[LightGBM] [Info] Number of positive: 11611, number of negative: 11611
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001115 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1098
[LightGBM] [Info] Number of data points in the train set: 23222, number of used features: 38
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Best F1-score (fraud class): 0.24177656229698838
Best parameters:
{'voting__xgb__n_estimators': 200, 'voting__xgb__learning_rate': 0.01, 'voting__rf__n_estimators': 100, 'voting__rf__max_depth': 10, 'voting__lgb__n_estimators': 200, 'voting__lgb__learning_rate': 0.01}
ROC AUC Score: 0.7987
[[2129  756]
 [  66  133]]
              precision    recall  f1-score   support

           0      0.970     0.738     0.838      2885
           1      0.150     0.668     0.244       199

    accuracy   

In [18]:
# Base models with class weightings
base_models = [
    ('lr', make_pipeline(StandardScaler(), LogisticRegression(max_iter=2000, class_weight="balanced"))),
    ('rf', RandomForestClassifier(n_estimators=100, class_weight="balanced", random_state=36)),
    ('xgb', xgb.XGBClassifier(n_estimators=100, eval_metric="logloss", random_state=42, scale_pos_weight=12)),
    ('lgb', lgb.LGBMClassifier(n_estimators=100, class_weight='balanced', random_state=42))
]

# Stronger meta model
meta_model = RandomForestClassifier(n_estimators=100, class_weight="balanced", random_state=42)

# Stacking classifier (no oversampled data used here!)
stack_clf = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5,
    passthrough=True,
    n_jobs=-1
)

# Full pipeline with SMOTE
pipeline = Pipeline([
    ('smote', SMOTE(random_state=42)),
    ('stacking', stack_clf)
])

# StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Cross-validated predictions on training data (simulate test set)
y_proba_cv = cross_val_predict(pipeline, X_train, y_train, cv=skf, method='predict_proba', n_jobs=-1)[:, 1]

# Find best threshold
precisions, recalls, thresholds = precision_recall_curve(y_train, y_proba_cv)
f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-6)
best_threshold = thresholds[f1_scores.argmax()]
print(f"Best Threshold by F1: {best_threshold:.2f}, F1-score: {f1_scores.max():.3f}")

# Refit final model on full training data
pipeline.fit(X_train, y_train)

# Predict on real test set
y_proba_test = pipeline.predict_proba(X_test)[:, 1]
y_pred_test = (y_proba_test >= best_threshold).astype(int)

roc_auc = roc_auc_score(y_test, y_proba_test)
print(f"ROC AUC Score: {roc_auc:.4f}")

# Evaluation
print("==== Final Stacking Classifier with CV + SMOTE ====")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_test))
print("Classification Report:")
print(classification_report(y_test, y_pred_test, digits=3))

# Model Update Summary (Stacking Classifier)
# - Replaced training on oversampled data with StratifiedKFold cross-validation
# - Applied SMOTE inside each fold using a pipeline to avoid data leakage
# - Used a stronger meta-model (Random Forest instead of Logistic Regression)
# - Tuned the decision threshold based on the best F1-score from precision-recall curve
# These changes aim to improve fraud class (label=1) recall and F1-score while keeping evaluation realistic

Best Threshold by F1: 0.11, F1-score: 0.255
ROC AUC Score: 0.7733
==== Final Stacking Classifier with CV + SMOTE ====
Confusion Matrix:
[[2496  389]
 [ 122   77]]
Classification Report:
              precision    recall  f1-score   support

           0      0.953     0.865     0.907      2885
           1      0.165     0.387     0.232       199

    accuracy                          0.834      3084
   macro avg      0.559     0.626     0.569      3084
weighted avg      0.903     0.834     0.864      3084



In [19]:
# Assuming numeric_df is your dataset (already loaded)
# Prepare the data
X = numeric_df.drop(['FraudFound_P', 'PolicyNumber'], axis=1)
y = numeric_df['FraudFound_P']

# Split the data into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define RandomUnderSampler
under_sampler = RandomUnderSampler(random_state=42)

# Apply RandomUnderSampler to the training data to reduce the majority class size
X_train_under, y_train_under = under_sampler.fit_resample(X_train, y_train)

# Check the class distribution after undersampling
print("Class distribution after undersampling:")
print(pd.Series(y_train_under).value_counts())

# Train and evaluate a RandomForest model on the original dataset (without undersampling)
clf_original = RandomForestClassifier(random_state=42)
clf_original.fit(X_train, y_train)
y_pred_original = clf_original.predict(X_test)

# Train and evaluate a RandomForest model on the undersampled dataset
clf_under = RandomForestClassifier(random_state=42)
clf_under.fit(X_train_under, y_train_under)
y_pred_under = clf_under.predict(X_test)

print("\nPerformance on undersampled dataset:")
print(classification_report(y_test, y_pred_under, zero_division=0))

Class distribution after undersampling:
FraudFound_P
0    724
1    724
Name: count, dtype: int64

Performance on undersampled dataset:
              precision    recall  f1-score   support

           0       0.99      0.65      0.78      2885
           1       0.15      0.87      0.25       199

    accuracy                           0.66      3084
   macro avg       0.57      0.76      0.51      3084
weighted avg       0.93      0.66      0.75      3084

